#### Import all libraries

In [1]:
from __future__ import print_function
import os
import collections
import SimpleITK as sitk
import numpy as np
import six
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
import sys
import logging as logger
import time
import json
import os
import logging.config
import pandas as pd
import gc
import math

In [2]:
print("Before starting the entire loop. First Check what is the maximum size of the rock you can run the code for...")

Before starting the entire loop. First Check what is the maximum size of the rock you can run the code for...


#### Automate the GLSZM feature extraction process

In [3]:
%%time
print("Starting the automation loop for GLSZM feature extraction")

# All the greyscale images of various porous media types are named as data#.nrrd. 
# Iterate over the names to calculate the GLSZM features
for num_img in range(19, 20):
    
    # Create the variables for image and label
    global image_1, label_1

    # Path of the images used in REV analysis
    path ='D:\\Ankita PhD\\pyradiomics\\data\\experiments\\data{}.nrrd'.format(num_img)
    
    # Read the nrrd image using the SITK library. Nrrd images are 8-bit images with intensity values between 0 to 255
    image_1 = sitk.ReadImage(path) 
    
    # Set the settings paramter needed for the bin_width and label
    # Bin_Width = 1 (as I don't want any further reduction in greyscale values and want to preserve the information)
    # label is a nrrd image that incorporates the window size and hence, gives a ROI for the GLSZM calculation.
    # The minimum size is set by the user and the mavimum size is the size of the image
    settings = {}
    settings['label'] = 255
    settings['binWidth'] = 1

    # Set-up logging for all the calculations
    log_file = 'D:\\Ankita PhD\\pyradiomics\\log_messages\\log{}.txt'.format(num_img)
    handler = logging.FileHandler(filename=log_file, mode='w')  # overwrites log_files from previous runs. Change mode to 'a' to append.
    formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")  # format string for log messages
    handler.setFormatter(formatter)
    radiomics.logger.addHandler(handler)

    # Control the amount of logging stored by setting the level of the logger. N.B. if the level is higher than the
    # Verbositiy level, the logger level will also determine the amount of information printed to the output
    radiomics.logger.setLevel(logging.DEBUG)

    # the size of the mask (same as label) should be the same as that of the image
    mask_x = image_1.GetWidth()
    mask_y = image_1.GetWidth()
    mask_z = image_1.GetWidth()

    #list for all the features being extracted
    v1 = []
    v2 = []
    v3 = []
    v4 = []
    v5 = []
    v6 = []
    v7 = []
    v8 = []
    v9 = []
    v10 = []
    v11 = []
    v12 = []
    v13 = []
    v14 = []
    v15 = []
    v16 = []

    print("Starting the domain centre calculcation for image",str(num_img))
    
    # here the window_size refers to the region of mask that will be used to segment the ROI from the image. The window
    # sizes increase with every iteration
    u_window_size = [2, 3, 4, 5, 6, 7, 8 , 9, 10, 20, 30, 40, 50, 60, 90, 100, 120, 150, 180, 200]#, 250, 275, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800] #200, 150, 100, 60, 50, 40, 30, 20, 10]# 650, 600, 550, 500, 450, 400, 350, 300, 250, 200, 150, 100, 60, 50, 40, 30, 20, 10] #10, 20, 30, 40, 50, 60, 100, 150, 200, 250, 300, 350, 400, 450, 600, 650]
    global window_size, total_iter, expect_iter, i_loc, j_loc, z_loc, bb, correctedMask, croppedImage, croppedMask, glcm_window
    
    for window_size in u_window_size:  

        i_loc = int((mask_x - window_size)/2)
        j_loc = int((mask_y - window_size)/2)
        k_loc = int((mask_z - window_size)/2)

        # Generate the mask
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[i_loc:i_loc+window_size,j_loc:j_loc+window_size,k_loc:k_loc+window_size] = 255
        #print("Window Location", str(i_loc), str(i_loc+window_size), str(j_loc), str(j_loc+window_size), str(k_loc), str(k_loc+window_size))
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        label_1.CopyInformation(image_1)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        print("Cropped Mask and Image created for domain centre size =",str(window_size))
        #calculate secondorder stats
        glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
        glszmFeatures.enableAllFeatures()
        print('Enabled GLSZM features...',)
        result = glszmFeatures.execute()

        print('Calculated GLSZM features: ')
        for (key, val) in six.iteritems(result):
            if key =='GrayLevelNonUniformity':
                v1.append(val)
            if key =='GrayLevelNonUniformityNormalized':
                v2.append(val)
            if key =='GrayLevelVariance':
                v3.append(val)
            if key =='HighGrayLevelZoneEmphasis':
                v4.append(val)
            if key == 'LargeAreaEmphasis':
                v5.append(val)
            if key == 'LargeAreaHighGrayLevelEmphasis':
                v6.append(val)
            if key == 'LargeAreaLowGrayLevelEmphasis':
                v7.append(val)
            if key == 'LowGrayLevelZoneEmphasis':
                v8.append(val) 
            if key == 'SizeZoneNonUniformity':
                v9.append(val)
            if key == 'SizeZoneNonUniformityNormalized':
                v10.append(val)
            if key == 'SmallAreaEmphasis':
                v11.append(val)
            if key == 'SmallAreaHighGrayLevelEmphasis':
                v12.append(val)
            if key == 'SmallAreaLowGrayLevelEmphasis':
                v13.append(val)
            if key == 'ZoneEntropy':
                v14.append(val)
            if key == 'ZonePercentage':
                v15.append(val)
            if key == 'ZoneVariance':
                v16.append(val)

        print("Completed calculations for domain centre size =", str(window_size))
    
    # Create a data-frame to store all the GLSZM features for the image    
    my_df = pd.DataFrame(list(zip(u_window_size, v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16)))
    my_df = my_df.rename(index = str, columns={0: "Window Sizes", 1: "GrayLevelNonUniformity", 2: "GrayLevelNonUniformityNormalized", \
                                   3: "GrayLevelVariance", 4: "HighGrayLevelZoneEmphasis", 5: "LargeAreaEmphasis", \
                                   6: "LargeAreaHighGrayLevelEmphasis", 7: "LargeAreaLowGrayLevelEmphasis", 8: "LowGrayLevelZoneEmphasis", \
                                   9: "SizeZoneNonUniformity", 10: "SizeZoneNonUniformityNormalized", 11: "SmallAreaEmphasis", \
                                   12: "SmallAreaHighGrayLevelEmphasis", 13: "SmallAreaLowGrayLevelEmphasis", 14: "ZoneEntropy", \
                                   15: "ZonePercentage", 16: "ZoneVariance"})    
    
    name_csv = 'glszm_dc_data{}.csv'.format(num_img)
    my_df.to_csv(name_csv, index=False, header=True)
    
    print("Stored all the GLSZM features for image", str(num_img))
    
    #free the memory and delete all the variables
    del image_1
    del my_df
    del glszmFeatures
    del result

Starting the automation loop for GLSZM feature extraction
Starting the domain centre calculcation for image 19
Cropped Mask and Image created for domain centre size = 2
Enabled GLSZM features...
Calculated GLSZM features: 
Completed calculations for domain centre size = 2
Cropped Mask and Image created for domain centre size = 3
Enabled GLSZM features...
Calculated GLSZM features: 
Completed calculations for domain centre size = 3
Cropped Mask and Image created for domain centre size = 4
Enabled GLSZM features...
Calculated GLSZM features: 
Completed calculations for domain centre size = 4
Cropped Mask and Image created for domain centre size = 5
Enabled GLSZM features...
Calculated GLSZM features: 
Completed calculations for domain centre size = 5
Cropped Mask and Image created for domain centre size = 6
Enabled GLSZM features...
Calculated GLSZM features: 
Completed calculations for domain centre size = 6
Cropped Mask and Image created for domain centre size = 7
Enabled GLSZM feature